**VIDEO FRAME EXTRACTION FOR INPUT VIDEO**

In [6]:
import cv2
import os
count=0
success=True
video=cv2.VideoCapture('InputVideo.mp4')
if not video.isOpened():
    print("Error opening video file!")
directory = "InputFrames"
parent_dir = "E:\saintgits folder\T (Mini Project)\CODE"
path = os.path.join(parent_dir, directory)
os.mkdir(path)
while success:
    success,image=video.read()
    if success:
        cv2.imwrite("InputFrames/frame%d.jpg" % count,image)
        count+=1

**VIDEO FRAME EXTRACTION FOR VIDEOS IN THE DATABASE**

In [7]:
n=int(input("Enter the number of videos in the DataBase: "))

def split(x):
  count=0
  success=True
  video=cv2.VideoCapture('Database/Video%d.mp4' % x)
  if not video.isOpened():
    print("Error opening database video file : Video%d!" % x)
  print("Currently Processing -> Video",x,": \n")

  directory = "Temp%d" % x
  parent_dir = "E:\saintgits folder\T (Mini Project)\CODE"
  path = os.path.join(parent_dir, directory)
  os.mkdir(path)

  while success:
    success,image=video.read()
    if success:
      cv2.imwrite("E:\saintgits folder\T (Mini Project)\CODE\Temp%d/frame%d.jpg" % (x,count),image)
      count+=1
    else:
      break

for i in range(n):
  split(i)

Currently Processing -> Video 0 : 

Currently Processing -> Video 1 : 

Currently Processing -> Video 2 : 

Currently Processing -> Video 3 : 



**RESNET AND COSINE SIMILARITY MEASURE FOR INPUT VIDEO FRAMES AND VIDEOS IN THE DATABASE**

In [9]:
import numpy as np
import os
import pandas as pd
from PIL import Image
import torch
import torchvision.models as models
import torchvision.transforms as transforms

resnet18 = models.resnet18(pretrained=True)

feature_extractor = torch.nn.Sequential(*list(resnet18.children())[:-1])

def extract_features(image_path, feature_extractor):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                         std=[0.229, 0.224, 0.225])])
    image = transform(image).unsqueeze(0)

    with torch.no_grad():
        features = feature_extractor(image).squeeze()

    return features.numpy()

def compare_images(image1_path, image2_path, feature_extractor):
    features1 = extract_features(image1_path, feature_extractor)
    features2 = extract_features(image2_path, feature_extractor)

    similarity = np.dot(features1, features2) / (np.linalg.norm(features1) * np.linalg.norm(features2))
    return similarity

>Reading the Cosine Similarities into a .csv file

In [16]:
image1_path = 'E:\saintgits folder\T (Mini Project)\CODE\InputFrames\\frame61.jpg'

for v in range(4):
    print("\n\nVideo%d:\n\n" % v)

    dir_path = r'E:\saintgits folder\\T (Mini Project)\\CODE\\Temp%d' % v
    n=(len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))]))
    
    L=[]

    print("Done With:\n")
    
    for i in range(n):
        print(i)
        image2_path='E:\saintgits folder\T (Mini Project)\CODE\Temp%d\\frame%d.jpg' % (v,i)
        L.append([v,compare_images(image1_path, image2_path, feature_extractor)])

    df = pd.DataFrame(L,columns=['Video','Similarity'])
    if v==0:
        df.to_csv('CosSimData.csv',index=False)
    else:
        df.to_csv('CosSimData.csv', mode='a', index=False, header=False)



Video0:


Done With:

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
27

In [23]:
df=pd.read_csv('CosSimData.csv')
print(df.loc[df['Similarity'] == df['Similarity'].max()])

      Video  Similarity
3825      2    0.997539


___

>Color Histogram

In [1]:
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to extract color histograms as features from a video
def extract_color_histograms(video_path, bins=(8, 8, 8)):
    cap = cv2.VideoCapture(video_path)
    features = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        features.append(hist)
    cap.release()
    return np.array(features)

# Function to calculate video similarity using color histograms
def calculate_video_similarity(video1_path, video2_path):
    vf1 = extract_color_histograms(video1_path)
    vf2 = extract_color_histograms(video2_path)
    similarity_matrix = cosine_similarity(vf1, vf2)
    average_similarity = np.mean(similarity_matrix)
    return average_similarity

# Example usage: check similarity between two videos
video1_path = 'E:\saintgits folder\T (Mini Project)\CODE\InputVideo.mp4'
video2_path = 'E:\saintgits folder\T (Mini Project)\CODE\Database\Video0.mp4'
similarity_score = calculate_video_similarity(video1_path, video2_path)
print("Video similarity score:", similarity_score)


Video similarity score: 0.45822707


In [3]:
import cv2
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Function to extract color histograms as features from a video
def extract_color_histograms(video_path, bins=(8, 8, 8)):
    cap = cv2.VideoCapture(video_path)
    features = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv], [0, 1, 2], None, bins, [0, 180, 0, 256, 0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        features.append(hist)
    cap.release()
    return np.array(features)

# Function to calculate video similarity using color histograms
def calculate_video_similarity(video1_path, video2_path):
    vf1 = extract_color_histograms(video1_path)
    vf2 = extract_color_histograms(video2_path)
    similarity_matrix = cosine_similarity(vf1, vf2)
    average_similarity = np.mean(similarity_matrix)
    return average_similarity

# Example usage: check similarity between two videos
video1_path = 'E:\saintgits folder\T (Mini Project)\CODE\InputVideo.mp4'

number=int(input("Enter the number of videos in the DB: "))

for i in range(number):
    print(i,":\t")
    video2_path = 'E:\saintgits folder\T (Mini Project)\CODE\Database\Video%d.mp4' % i
    similarity_score = calculate_video_similarity(video1_path, video2_path)
    print("Video similarity score:", similarity_score)


0 :	
Video similarity score: 0.45822707
1 :	
Video similarity score: 0.06255585
2 :	
Video similarity score: 0.061472297
3 :	
Video similarity score: 0.31092092


>Structural Similarity Index Measure

In [6]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

# Function to calculate SSIM between two videos
def calculate_ssim(video1, video2):
    cap1 = cv2.VideoCapture(video1)
    cap2 = cv2.VideoCapture(video2)

    ssim_scores = []

    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        if not ret1 or not ret2:
            break
        
        (H, W, C) = frame1.shape
        frame2 = cv2.resize(frame2, (W, H))

        # Convert frames to grayscale
        frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

        # Calculate SSIM between frames
        score = ssim(frame1_gray, frame2_gray)

        ssim_scores.append(score)

    cap1.release()
    cap2.release()

    # Calculate average SSIM score
    avg_ssim = np.mean(ssim_scores)

    return avg_ssim

# Example usage
video1 = "E:\saintgits folder\T (Mini Project)\CODE\InputVideo.mp4"

number=int(input("Enter the number of videos in the DB: "))

for i in range(number):
    print(i,":\t")
    video2= 'E:\saintgits folder\T (Mini Project)\CODE\Database\Video%d.mp4' % i
    similarity_score = calculate_ssim(video1, video2)
    print("Similarity score:", similarity_score)


0 :	
Similarity score: 0.26838284823082464
1 :	
Similarity score: 0.24721450828501648
2 :	
Similarity score: 0.15389513066717894
3 :	
Similarity score: 0.1633543222570141


_____________

>LSH Indexing

In [17]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

# Load the pre-trained ResNet50 model
resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

# Load and process video frames
video_path = "E:\saintgits folder\T (Mini Project)\CODE\InputVideo.mp4"
video_capture = cv2.VideoCapture(video_path)

frame_features = []
while video_capture.isOpened():
    ret, frame = video_capture.read()
    if not ret:
        break

    # Resize frame to the input size required by ResNet50
    resized_frame = cv2.resize(frame, (224, 224))

    # Preprocess the frame
    preprocessed_frame = preprocess_input(resized_frame)

    # Expand dimensions to match the input shape of the model
    expanded_frame = np.expand_dims(preprocessed_frame, axis=0)

    # Extract features using ResNet50
    features = resnet_model.predict(expanded_frame)

    frame_features.append(features.flatten())

video_capture.release()

# Convert the frame-level features to a numpy array
frame_features = np.array(frame_features)


1/1 [==============================] - 0s 104ms/step


In [18]:
print(frame_features.shape)

(152, 2048)


>LSH Indexing function (takes too much time: 39 mins)

In [54]:
def extract_features(video_path):
    # Load the pre-trained ResNet50 model
    resnet_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

    # Load and process video frames
    video_capture = cv2.VideoCapture(video_path)

    frame_features = []
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        # Resize frame to the input size required by ResNet50
        resized_frame = cv2.resize(frame, (224, 224))

        # Preprocess the frame
        preprocessed_frame = preprocess_input(resized_frame)

        # Expand dimensions to match the input shape of the model
        expanded_frame = np.expand_dims(preprocessed_frame, axis=0)

        # Extract features using ResNet50
        features = resnet_model.predict(expanded_frame)

        frame_features.append(features.flatten())

    video_capture.release()

    # Convert the frame-level features to a numpy array
    frame_features = np.array(frame_features)
    # Define the LSTM model
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(units=50, input_shape=frame_features.shape))
    model.add(tf.keras.layers.Dense(units=100))

    # Compile the model
    model.compile(loss='mse', optimizer='adam')

    # Reshape frame-level features to match LSTM input shape
    frame_features_reshaped = np.reshape(frame_features, (1, frame_features.shape[0], frame_features.shape[1]))

    # Combine frame-level features into a video-level representation
    video_rep = model.predict(frame_features_reshaped)
    return video_rep

def lsh_indexing(input_video,db_path,num):
    input_rep=extract_features(input_video)
    video_rep=[]
    for i in range(num):
        video_path = db_path+'\\Video%d.mp4' % i
        video_features=extract_features(video_path)
        video_rep.append(video_features)

    video_rep=np.array(video_rep)

inputt="E:\saintgits folder\T (Mini Project)\CODE\InputVideo.mp4"
dbb="E:\saintgits folder\T (Mini Project)\CODE\Database"

lsh_indexing(inputt,dbb,4)

1/1 [==============================] - 0s 346ms/step


In [22]:
import numpy as np
import tensorflow as tf

# Assuming you have frame-level features stored in a 3D array
#frame_features = ...  # Shape: (num_frames, sequence_length, feature_dim)

# Define the LSTM model
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units=50, input_shape=(152, 2048)))
model.add(tf.keras.layers.Dense(units=100))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Reshape frame-level features to match LSTM input shape
frame_features_reshaped = np.reshape(frame_features, (1, frame_features.shape[0], frame_features.shape[1]))

# Combine frame-level features into a video-level representation
video_representation = model.predict(frame_features_reshaped)
print(video_representation)


1/1 [==============================] - 0s 280ms/step
[[-3.2132886e-02 -2.6289570e-01 -8.3833307e-02 -2.1374443e-01
  -6.2528662e-02 -4.7349480e-01 -3.1608734e-01  2.7009499e-01
   1.1085601e-01  4.4545904e-01 -2.2842675e-01 -2.6916844e-01
  -8.5069075e-02  1.8589479e-01 -5.1981705e-01 -1.8266483e-01
  -3.6521647e-02  5.0836116e-02 -3.7336153e-01 -5.8824152e-01
   4.4180650e-01 -1.6100718e-01 -2.8050038e-01 -1.5305284e-01
  -3.8683957e-01 -5.1493275e-01 -1.8885186e-01  5.2979666e-01
  -2.7838740e-01  3.9378833e-03 -7.4738216e-01  4.5881975e-01
  -1.4353719e-01  6.6701543e-01  5.3187702e-02  5.4094815e-01
  -2.0217496e-01  7.5502828e-02 -7.0600462e-01 -1.4828220e-01
   2.7611321e-01 -1.4393680e-01  1.4573407e-01  1.0314648e-02
  -1.5954892e-01 -1.9804327e-02  1.9924816e-01  2.1117154e-01
  -6.4753187e-01  6.8549532e-01 -1.8411088e-01  3.4350768e-01
  -1.6668032e-01 -8.2141690e-02 -4.3697125e-01 -1.0431402e-01
   2.4735484e-01 -6.2915720e-02  5.0338149e-02 -4.0318269e-02
  -8.1192069e-02 

In [23]:
print(video_representation.shape)

(1, 100)


In [42]:
input_path = "E:\saintgits folder\T (Mini Project)\CODE\InputVideo.mp4"
input_capture = cv2.VideoCapture(video_path)

query_features = []
while input_capture.isOpened():
    ret, frame = input_capture.read()
    if not ret:
        break

    # Resize frame to the input size required by ResNet50
    resized_frame = cv2.resize(frame, (224, 224))

    # Preprocess the frame
    preprocessed_frame = preprocess_input(resized_frame)

    # Expand dimensions to match the input shape of the model
    expanded_frame = np.expand_dims(preprocessed_frame, axis=0)

    # Extract features using ResNet50
    features = resnet_model.predict(expanded_frame)

    query_features.append(features.flatten())

input_capture.release()

# Convert the frame-level features to a numpy array
query_features = np.array(frame_features)

1/1 [==============================] - 0s 104ms/step


In [45]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units=50, input_shape=(152, 2048)))
model.add(tf.keras.layers.Dense(units=100))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Reshape frame-level features to match LSTM input shape
query_features_reshaped = np.reshape(query_features, (1, query_features.shape[0], query_features.shape[1]))

# Combine frame-level features into a video-level representation
query_video = model.predict(query_features_reshaped)


1/1 [==============================] - 0s 320ms/step


In [49]:
# Initialize LSH index
(v,d)=video_representation.shape
nbits=d*4
index = faiss.IndexLSH(d, nbits)  # d is the dimensionality of the video representation

# Add video representations to the index
index.add(video_representation)

# Step 4: Query processing
# Extract features from query video
database_videos=np.array([0,1,2,3])

# Search for similar videos
k = 10  # Number of nearest neighbors to retrieve
distances, indices = index.search(query_video, 1)

# Retrieve similar video paths from the indices
similar_videos = database_videos[indices[0]]

# Print the paths of similar videos
for video_path in similar_videos:
    print(video_path)

0
